In [3]:
%run ../../Utils/yp_utils.py

# Initial setup

In [4]:
paper_pmid = 31885205
paper_name = 'galardini_beltrao_2019' 

In [5]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [6]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [12]:
original_data = pd.read_csv('raw_data/Table_EV2_ko_scores.txt', sep='\t', skiprows=2)

In [15]:
original_data = original_data.loc[original_data['strain']=='S288C',:]

In [16]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 221555 x 7


In [21]:
original_data2 = original_data.groupby(['condition','gene'])['score'].mean().reset_index()

In [23]:
original_data3 = pd.pivot_table(original_data2, index='gene', columns='condition', values='score')

In [24]:
original_data3.shape

(4889, 38)

In [33]:
original_data3['orfs'] = original_data3.index.values.astype(str)

In [34]:
# Eliminate all white spaces & capitalize
original_data3['orfs'] = clean_orf(original_data3['orfs'])

In [35]:
# Fix typos
original_data3.loc[original_data3['orfs']=='YLR287-A','orfs'] = 'YLR287C-A'

In [36]:
# Translate to ORFs 
original_data3['orfs'] = translate_sc(original_data3['orfs'], to='orf')

In [37]:
# Make sure everything translated ok
t = looks_like_orf(original_data3['orfs'])
print(original_data3.loc[~t,])
# Output: found 22 inexisting ORFs and WT

condition    2,4-Dichlorophenoxyacetic acid (48H)  39ºC (48H)  39ºC (72H)  \
index_input                                                                 
WT                                       0.106202    0.224502    0.285540   
YAR037W                                  1.502639   -0.487772   -0.136703   
YAR040C                                 -2.018356    0.233355   -0.061323   
YAR043C                                 -0.269313   -0.335730    0.086679   
YCL006C                                  0.436132   -0.152394    0.108633   
YCL013W                                 -0.351663    0.410026   -0.089002   
YCL026C                                 -0.486873   -0.600623    0.095393   
YCL074W                                  0.211112    0.278586    0.058407   
YCL075W                                  0.132772    0.398907    0.173588   
YDR134C                                  0.353653    0.826668    0.441954   
YER108C                                  1.520707   -0.842644   -0.186131   

In [38]:
original_data3 = original_data3.loc[t,:]

In [39]:
original_data3.set_index('orfs', inplace=True)
original_data3.index.name='orf'

In [40]:
original_data3 = original_data3.groupby(original_data3.index).mean()

In [41]:
original_data3.shape

(4838, 38)

# Load dataset ids

In [62]:
dataset_id_map = pd.read_excel('raw_data/msb198831-sup-0002-tableev1.xlsx', encoding='utf-8',
                               sheet_name='Sheet1', header=None)

In [63]:
dataset_id_map[0] = dataset_id_map[0].apply(lambda x: x.replace('Â',''))

In [65]:
dataset_id_map.set_index(0, inplace=True)

In [66]:
dataset_id_map = dataset_id_map.reindex(index=original_data3.columns.values)

In [68]:
dataset_ids = list(dataset_id_map[1])

# Prepare the final dataset

In [70]:
data = original_data3.copy()

In [71]:
datasets = datasets.reindex(index=dataset_ids)

In [72]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [73]:
data.head()

dataset_id,16349,16351,16385,16353,16354,16355,16386,16356,16357,16358,...,16390,16374,16391,16375,16382,16377,16392,16384,16378,16383
data_type,value,value,value,value,value,value,value,value,value,value,...,value,value,value,value,value,value,value,value,value,value
orf,,,,,,,,,,,,,,,,,,,,,
YAL002W,-6.027977,-1.966362,-1.437009,-1.945708,1.345415,-3.148804,-3.282090,-2.645313,0.198431,-0.647809,...,0.982522,-0.750200,-0.634745,2.735846,-0.017735,0.823463,1.020318,1.714669,1.396525,-0.536990
YAL004W,0.086213,0.294230,0.897366,-0.816921,0.591333,0.509435,0.572919,1.242551,0.541600,-0.459679,...,-0.716532,-0.339391,-0.160671,-1.064505,1.808986,-1.166473,-0.948275,1.365731,1.105575,-0.052912
YAL005C,-0.875837,-0.952093,-0.462713,0.930383,0.900128,-1.521974,-1.627128,-0.263197,0.075423,-0.121187,...,-0.603338,-0.585939,-0.223783,-0.196172,0.870998,-0.600152,-0.427590,0.175162,1.375118,2.872251
YAL007C,0.118975,0.708104,0.424890,-0.318803,-1.143170,-0.062520,-0.197061,0.126095,0.152684,0.253159,...,0.578599,-0.029965,-0.172207,0.338149,0.025946,-0.074581,0.055680,-0.118836,-1.090361,0.032435
YAL008W,1.246915,0.863167,0.842147,-0.182611,-0.067659,0.387749,0.256179,0.012162,0.710225,0.298724,...,-0.212906,-0.356066,-0.638409,0.290235,-0.151671,0.126812,0.102794,-1.002664,-1.455795,-0.487320


## Subset to the genes currently in SGD

In [74]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 1


In [75]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [76]:
data.head()

,dataset_id,16349,16351,16385,16353,16354,16355,16386,16356,16357,16358,...,16390,16374,16391,16375,16382,16377,16392,16384,16378,16383
,data_type,value,value,value,value,value,value,value,value,value,value,...,value,value,value,value,value,value,value,value,value,value
gene_id,orf,,,,,,,,,,,,,,,,,,,,,
2,YAL002W,-6.027977,-1.966362,-1.437009,-1.945708,1.345415,-3.148804,-3.282090,-2.645313,0.198431,-0.647809,...,0.982522,-0.750200,-0.634745,2.735846,-0.017735,0.823463,1.020318,1.714669,1.396525,-0.536990
1863,YAL004W,0.086213,0.294230,0.897366,-0.816921,0.591333,0.509435,0.572919,1.242551,0.541600,-0.459679,...,-0.716532,-0.339391,-0.160671,-1.064505,1.808986,-1.166473,-0.948275,1.365731,1.105575,-0.052912
4,YAL005C,-0.875837,-0.952093,-0.462713,0.930383,0.900128,-1.521974,-1.627128,-0.263197,0.075423,-0.121187,...,-0.603338,-0.585939,-0.223783,-0.196172,0.870998,-0.600152,-0.427590,0.175162,1.375118,2.872251
5,YAL007C,0.118975,0.708104,0.424890,-0.318803,-1.143170,-0.062520,-0.197061,0.126095,0.152684,0.253159,...,0.578599,-0.029965,-0.172207,0.338149,0.025946,-0.074581,0.055680,-0.118836,-1.090361,0.032435
6,YAL008W,1.246915,0.863167,0.842147,-0.182611,-0.067659,0.387749,0.256179,0.012162,0.710225,0.298724,...,-0.212906,-0.356066,-0.638409,0.290235,-0.151671,0.126812,0.102794,-1.002664,-1.455795,-0.487320


# Normalize

In [77]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [78]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [79]:
data_norm[data.isnull()] = np.nan

In [80]:
data_all = data.join(data_norm)

In [81]:
data_all.head()


,dataset_id,16349,16351,16385,16353,16354,16355,16386,16356,16357,16358,...,16390,16374,16391,16375,16382,16377,16392,16384,16378,16383
,data_type,value,value,value,value,value,value,value,value,value,value,...,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,,,,,,,,,,
2,YAL002W,-6.027977,-1.966362,-1.437009,-1.945708,1.345415,-3.148804,-3.282090,-2.645313,0.198431,-0.647809,...,1.556762,-0.920472,-0.822961,2.624789,0.000356,1.475570,1.441070,1.748657,1.263195,-0.363666
1863,YAL004W,0.086213,0.294230,0.897366,-0.816921,0.591333,0.509435,0.572919,1.242551,0.541600,-0.459679,...,-1.066656,-0.304253,-0.158967,-1.090058,2.242069,-1.286525,-1.221271,1.407881,0.961066,-0.065056
4,YAL005C,-0.875837,-0.952093,-0.462713,0.930383,0.900128,-1.521974,-1.627128,-0.263197,0.075423,-0.121187,...,-0.891879,-0.674078,-0.247362,-0.241262,1.090991,-0.500453,-0.517092,0.245158,1.240964,1.739368
5,YAL007C,0.118975,0.708104,0.424890,-0.318803,-1.143170,-0.062520,-0.197061,0.126095,0.152684,0.253159,...,0.933087,0.159889,-0.175125,0.281037,0.053960,0.229056,0.136485,-0.041964,-1.319244,-0.012409
6,YAL008W,1.246915,0.863167,0.842147,-0.182611,-0.067659,0.387749,0.256179,0.012162,0.710225,0.298724,...,-0.289034,-0.329266,-0.828093,0.234202,-0.164008,0.508596,0.200203,-0.905120,-1.698720,-0.333026


# Print out

In [82]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [83]:
from IO.save_data_to_db3 import *

In [84]:
save_data_to_db(data_all, paper_pmid)

Deleting all datasets for PMID 31885205...


  0%|          | 0/38 [00:00<?, ?it/s]

Inserting the new data...


100%|██████████| 38/38 [04:39<00:00,  7.35s/it]

Updating the data_modified_on field...
